<a href="https://colab.research.google.com/github/narendranathe/Web-Scraping/blob/main/Nixon_scrape_Yelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install selenium

In [5]:
# Import all necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import openpyxl

# Set up Chrome options for Selenium WebDriver
chrome_options = Options()
chrome_options.add_argument('--headless')  # Enable headless mode for background execution
chrome_options.add_argument('--no-sandbox')  # Bypass OS security model, required on Linux if running as root
chrome_options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
d = webdriver.Chrome(options=chrome_options)

# Initialize variables
i = 0
all_reviews = []

# Loop to paginate through the Yelp reviews
while True:
    # Construct the URL with pagination
    url = "https://www.yelp.com/biz/richard-nixon-presidential-library-and-museum-yorba-linda-2?start=" + str(i)
    i += 10  # Increment to fetch the next page of reviews
    d.get(url)  # Navigate to the page
    soup = BeautifulSoup(d.page_source, 'html.parser')  # Parse the page source with BeautifulSoup

    # Find the main reviews container
    temp = soup.find('div', {'class': 'css-1qn0b6x', 'id': 'reviews'})
    temp = temp.find('ul', class_='list__09f24__ynIEd') if temp else None
    reviews = temp.find_all('li', class_='css-1q2nwpv') if temp else []
    if len(reviews) < 2:  # Break the loop if there are less than two reviews, indicating end of reviews
        break

    # Loop through each review and extract details
    for review in reviews:
        # Extract user details
        userdetails = review.find('div', class_='user-passport-info')
        name = userdetails.find('a', class_='css-19v1rkv').text if userdetails.find('a', class_='css-19v1rkv') else "Not found"
        elite = userdetails.find('div', class_='elite-badge__09f24__dykWK').find('span', class_='css-1adhs7a').text if userdetails.find('div', class_='elite-badge__09f24__dykWK') else "Not found"
        location = userdetails.find('span', class_='css-qgunke').text if userdetails.find('span', class_='css-qgunke') else "Not found"

        # Extract user stats
        userstats = review.find('div', {'class':'user-passport-stats__09f24__NQxB4'})
        friends = userstats.find('div', {'aria-label':'Friends'}).text if userstats.find('div', {'aria-label':'Friends'}) else "Not found"
        review_count = userstats.find('div', {'aria-label':'Reviews'}).text if userstats.find('div', {'aria-label':'Reviews'}) else "Not found"
        pictures = userstats.find('div', {'aria-label':'Photos'}).text if userstats.find('div', {'aria-label':'Photos'}) else "Not found"

        # Extract review details
        ratings = review.find('div', class_='five-stars__09f24__mBKym')
        ratings_text = ratings['aria-label'] if ratings and 'aria-label' in ratings.attrs else "Not found"
        review_date = review.find('span', class_='css-chan6m').text if review.find('span', class_='css-chan6m') else "Not found"
        is_review_updated = True if review.find('span', class_='reviewQualifier__09f24__iY9mB') else False

        # Extract additional information
        extra = review.find('div', class_='margin-b2__09f24__CEMjT')
        photos = extra.find('a', class_='css-19v1rkv').text if extra and extra.find('a', class_='css-19v1rkv') else "Not found"
        check_ins = extra.find('span', class_='display--inline__09f24__c6N_k').text if extra and extra.find('span', class_='display--inline__09f24__c6N_k') else "Not found"
        comment = review.find('p', class_='comment__09f24__D0cxf').text if review.find('p', class_='comment__09f24__D0cxf') else "Not found"

        # Extract review reactions
        reactions = [tt.text for tt in review.find_all('span', class_='css-1xfc281')]

        # Store the extracted data in a list
        review_data = [name, elite, location, friends, review_count, pictures, ratings_text, review_date, is_review_updated, photos, check_ins, comment, str(reactions)]
        all_reviews.append(review_data)

# Quit the WebDriver session
d.quit()

# Save the scraped data to a text file
with open("yelp_reviews.txt", "w", encoding="utf-8") as txt_file:
    for review_data in all_reviews:
        txt_file.write("\t".join(map(str, review_data)) + "\n")

# Save the scraped data to an Excel file
wb = openpyxl.Workbook()
ws = wb.active
headers = ["name", "elite", "location", "friends", "review_count", "pictures", "ratings_text", "review_date", "is_review_updated", "photos", "check_ins", "comment", "reactions"]
ws.append(headers)  # Add headers to the Excel sheet
for review_data in all_reviews:
    ws.append(review_data)  # Add review data
wb.save("yelp_reviews.xlsx")  # Save the workbook
wb.close()  # Close the workbook


In [6]:
from google.colab import files

# Download the Excel file
files.download('yelp_reviews.xlsx')

# Download the text file
files.download('yelp_reviews.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>